<a href="https://colab.research.google.com/github/Vinshul44/Trading-System-with-ML-Automation/blob/main/Mainxgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Applying XGboost
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# If you don’t import from strategy.py, define this here
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def compute_macd(close, fast=12, slow=26, signal=9):
    ema_fast = close.ewm(span=fast, adjust=False).mean()
    ema_slow = close.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    macd_hist = macd - signal_line
    return macd, signal_line, macd_hist
def apply_strategy(df):
    df['RSI'] = compute_rsi(df['Close'], 14)
    df['20_MA'] = df['Close'].rolling(window=20).mean()
    df['50_MA'] = df['Close'].rolling(window=50).mean()
    df['Buy_Signal'] = (df['RSI'] < 30) & (df['20_MA'] > df['50_MA'])
    return df

def train_model(df):
    # Calculate indicators
    df['RSI'] = compute_rsi(df['Close'], 14)
    macd, signal_line, hist = compute_macd(df['Close'])
    df['MACD'] = macd
    df['20_MA'] = df['Close'].rolling(window=20).mean()
    # Handle missing 'Volume'
    if 'Volume' not in df.columns:
        df['Volume'] = 0

    # Create target
    df['Target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)

    features = ['RSI', 'MACD', 'Volume', '20_MA']
    target = 'Target'

    if df.empty:
        raise ValueError("No valid rows after dropping NaNs.")

    # Prepare training
    X = df[features]
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = XGBClassifier(use_label_encoder=True, eval_metric='logloss')
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)

    return model, acc


In [14]:
from data_fetch import fetch_stock_data
from backtest import backtest
from google_sheet import connect_google_sheet, log_trades
from telegram_bot import send_telegram
import warnings
warnings.filterwarnings('ignore')
import os

os.environ["GOOGLE_SHEETS_CREDENTIALS"] = "/content/credentials.json.json"
os.environ["TELEGRAM_TOKEN"] = "8143044630:AAEKt3jjfUnuiCIqh654CBjm9md8m7nv-So"
os.environ["TELEGRAM_CHAT_ID"] = "5417266671"
stock_list = ['RELIANCE.NS', 'TCS.NS', 'INFY.NS']
data = fetch_stock_data(stock_list, '2024-01-01', '2025-04-01')
sheet = connect_google_sheet("Algo_Trading_Log")


for stock, df in data.items():
    df = apply_strategy(df)

    if df.empty or 'Buy_Signal' not in df.columns:
        print(f"{stock} - No data or signal to process.")
        continue

    if not df['Buy_Signal'].any():
        print(f"{stock} - No buy signals in strategy.")
        continue

    try:
        trades = backtest(df)
    except ValueError as ve:
        print(f"{stock} - Backtest error: {ve}")
        continue

    # Convert pandas Series to native types for Google Sheets logging
    clean_trades = []
    for trade in trades:
        clean_trade = {k: (v.item() if hasattr(v, 'item') else v) for k, v in trade.items()}
        clean_trades.append(clean_trade)

    log_trades(sheet, clean_trades)
    send_telegram(f"{stock}: {len(trades)} trades logged.")

    if len(df.dropna()) > 60:
        model, acc = train_model(df)
        print(f"{stock} - ML Accuracy: {acc:.2f}")
    else:
        print(f"{stock} - Skipping ML model: not enough data")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


✅ Telegram alert sent!
RELIANCE.NS - ML Accuracy: 0.68
✅ Telegram alert sent!
TCS.NS - ML Accuracy: 0.42
✅ Telegram alert sent!
INFY.NS - ML Accuracy: 0.42
